# Introduction to Visualizaiton Libraries

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shinchu/dataviz-notebooks/blob/main/week_3/intro-to-altair.ipynb)

## Pythonの主要な可視化ライブラリ

* [matplotlib](https://matplotlib.org/): 最も広く使われている。かなり細かいところまで調整できるので、論文に載せる図はこれで作ることが多い。
* [seaborn](https://seaborn.pydata.org/): matplotlibのラッパー。matplotlibのデザインをデフォルトでいい感じにしてくれる。細かい調整は内部のmatplotlibにアクセスして行う。
* [plotly](https://plotly.com/python/): インタラクティブな図を作ることができる。さっと使う分にはとても便利。
* [altair](https://altair-viz.github.io/): 宣言型統計可視化ライブラリ。データと視覚記号の関係を宣言することで図を作ることができる。
* ...

※ 宣言型（declarative）: 何をするか（what）を書く

※ 命令型（imperetive）: どうするか（how）を書く

本講義では、matplotlibとaltairを主に扱います。手軽にインタラクティブな図を作れるplotlyも少し紹介します。

## Altair

In [7]:
# ライブラリのインストール

!pip install altair
!pip install vega_datasets


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 3.2 MB/s eta 0:00:007.3 MB/s eta 0:00:01

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import altair as alt

In [4]:
pd.set_option('display.max_rows', 300)

### データの読み込み

* AltairではPandasデータフレームを利用してデータの読み込みを行う
* データセット、URL、直接入力などでデータを与えることができる
* この時、データフレームの形が[Tidy data](http://vita.had.co.nz/papers/tidy-data.html)であることが望ましい（Tidy dataになっていない時は事前に形を整える）
    * Tidy dataでは、各変数が列、各観測値が行、各観測単位がテーブルとなっている
    * このような形を維持することでデータの操作、モデル化、可視化が容易となる

#### データセットから読み込む

In [5]:
from vega_datasets import data
cars = data.cars()
cars.head()

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA


#### URLから読み込む

In [6]:
data.cars.url

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/cars.json'

In [7]:
pd.read_json(data.cars.url).head()

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA


#### 直接入力

ここでは、都市と月の平均降雨量（`precip`）を含む簡単なデータフレームを扱います。

In [8]:
df = pd.DataFrame({
    'city': ['Seattle', 'Seattle', 'Seattle', 'New York', 'New York', 'New York', 'Chicago', 'Chicago', 'Chicago'],
    'month': ['Apr', 'Aug', 'Dec', 'Apr', 'Aug', 'Dec', 'Apr', 'Aug', 'Dec'],
    'precip': [2.68, 0.87, 5.31, 3.94, 4.13, 3.58, 3.62, 3.98, 2.56]
})

df

,city,month,precip
0,Seattle,Apr,2.68
1,Seattle,Aug,0.87
2,Seattle,Dec,5.31
3,New York,Apr,3.94
4,New York,Aug,4.13
5,New York,Dec,3.58
6,Chicago,Apr,3.62
7,Chicago,Aug,3.98
8,Chicago,Dec,2.56


### Chartオブジェクト

Altairの基本的なオブジェクトは`Chart`で、引数としてデータフレームを受け取ります。

In [9]:
chart = alt.Chart(df)

### 視覚記号・視覚変数・エンコーディング

Chartオブジェクトに指示を与えることで、どの視覚記号を使うかを指定します。

例えば、`mark_point()`で視覚記号として、点を使うことができます。

In [10]:
chart.mark_point()

/Users/shu/.virtualenvs/datatools/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

ここでは、データセット1行につき1つの点が描かれているはずですが、点の位置（視覚変数）がどのデータ変数に対応するかをまだ指定していないため、全ての点が重なっています。

点を分離するために、データ変数を視覚変数にマッピングしましょう。

例えば、データ変数`city`をY軸を表す視覚変数`y`にマッピングすることができます。これには`encode`メソッドを使います。

In [11]:
chart.mark_point().encode(
    y='city'
)

alt.Chart(...)

`encode()`メソッドは、視覚変数（`x, y, color, shape, size`など）とデータ変数との間にマッピングを構築します。Pandasデータフレームに対して、Altairは自動的に適切なデータタイプを割り当てることができます。上の例では自動的に名義尺度が使われています。

上図では、各カテゴリ内ではまだ複数の点が重なっているため、データ変数`precip`を視覚変数`x`にマッピングすることでこれらを分離します。

In [12]:
chart.mark_point().encode(
    x='precip',
    y='city'
)

alt.Chart(...)

上図から、降雨量が最も少ない月と最も多い月の両方がシアトルに現れていることが分かります。

今回も、`precip`のデータタイプはAltairによって自動的に推論され、量的尺度が使われています。そのため、グリッド線とX軸の数値が自動的に追加されました。

以上ではキーワード引数でマッピングを行ったが、Altairでは`alt.X('precip')`という構文を用いることもできます。この方法は、マッピングの際に更にパラメータを追加することができるため、便利です。

In [13]:
chart.mark_point().encode(
    alt.X('precip'),
    alt.Y('city')
)

alt.Chart(...)

これまではデータ変数の性質をPandasデータフレームの型に基づいて自動的に推論してきました。これを明示的に指定することもできます。この方が安全なので、データ変数の性質は明示することを心がけましょう。

- 'b:N': 名義尺度：順序付けされていないカテゴリデータ
- 'b:O': 順序尺度：順序付けされているデータ
- 'b:Q': 量的尺度：数値データ

例えば、`alt.X('precip:N')`のように指定します。

In [14]:
chart.mark_point().encode(
    alt.X('precip:Q'),
    alt.Y('city:N')
)

alt.Chart(...)

> precipを量的尺度ではなく、名義尺度や順序尺度として扱うと上図はどのようになるでしょうか？試してみましょう。

In [15]:
# your code goes here

### データ変換：集約

データ探索のためのより柔軟な可視化のために、Altairにはデータを集約するための構文があります。

例えば、データ名と一緒に集約関数`average`を指定すると全ての値の平均を算出することができます。

In [16]:
chart.mark_point().encode(
    x='average(precip)',
    y='city'
)

alt.Chart(...)

X軸の各カテゴリには、そのカテゴリ内の値の平均を示す1つのポイントが表示されています。

Altairでは、`count, min, max, average, median, stdev`などの集約関数を使うことができます。また、自分で関数を新たに書くこともできます。

### 視覚記号の変更

集約された値を点ではなく棒で表現したい場合は、`mark_bar()`を使います。

In [17]:
chart.mark_bar().encode(
    x='average(precip)',
    y='city'
)

alt.Chart(...)

縦の棒グラフにするには、`x`と`y`のキーワードを入れ替えるだけです。

In [18]:
chart.mark_bar().encode(
    x='city',
    y='average(precip)'
)

alt.Chart(...)

### 可視化のカスタマイズ

`Chart.mark_*`メソッドのプロパティを使って軸のタイトルを変更したり、スケールを変更したり、色を変更したりすることができます。

In [19]:
chart.mark_point(color='firebrick').encode(
  alt.X('precip',
        scale=alt.Scale(type='log'),
        axis=alt.Axis(title='Log-Scaled Values')),
  alt.Y('city',
        axis=alt.Axis(title='Category')),
)

alt.Chart(...)

### 複数のビュー

ビュー合成演算子（view composition operators）を使うことで、複数のチャートを組み合わせてより複雑な図を作成することができます。

例として、上で読み込んだ自動車のデータセットを使います。



In [20]:
cars.shape

(406, 9)

In [21]:
cars.head()

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA


In [22]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Name              406 non-null    object        
 1   Miles_per_Gallon  398 non-null    float64       
 2   Cylinders         406 non-null    int64         
 3   Displacement      406 non-null    float64       
 4   Horsepower        400 non-null    float64       
 5   Weight_in_lbs     406 non-null    int64         
 6   Acceleration      406 non-null    float64       
 7   Year              406 non-null    datetime64[ns]
 8   Origin            406 non-null    object        
dtypes: datetime64[ns](1), float64(4), int64(2), object(2)
memory usage: 28.7+ KB


まずは自動車データセットを製造年ごとの平均燃費を示す折線グラフを表示します。

In [23]:
alt.Chart(cars).mark_line().encode(
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)

/Users/shu/.virtualenvs/datatools/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

このプロットの平均化されたデータポイントに丸を表示することができます。

折線グラフと散布図の2つのグラフを別々に定義し、`layer`演算子を使って組み合わせます。ここでは`+`演算子を使いましょう。

In [24]:
line = alt.Chart(cars).mark_line().encode(
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)

point = alt.Chart(cars).mark_circle().encode(
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)

line + point

alt.LayerChart(...)

チャートの定義を再利用することもできます。ここでは折れ線グラフを定義した上で、`mark_circle`メソッドを呼び出しています。

In [25]:
mpg = alt.Chart(cars).mark_line().encode(
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)

mpg + mpg.mark_circle()

alt.LayerChart(...)

このグラフを他のグラフと並べたい場合は、別の演算子を使うことで実現することができます。

例えば、時間経過に伴う平均馬力の変化のグラフと並べたいとすると、`|`演算子を使うことで横に並べることができますし、`&`演算子を使うことで縦に並べることができます。

In [26]:
hp = alt.Chart(cars).mark_line().encode(
    alt.X('Year'),
    alt.Y('average(Horsepower)')
)

(mpg + mpg.mark_circle()) | (hp + hp.mark_circle())

alt.HConcatChart(...)

In [27]:
(mpg + mpg.mark_circle()) & (hp + hp.mark_circle())

alt.VConcatChart(...)

このデータセットでは、1970年代から80年代前半にかけて、車の平均燃費が向上する一方で、平均馬力が低下していることが分かります。

### インタラクション

基本的な可視化に加えて、Altairではインタラクティブな図を作ることができます。

`interactive`メソッドを呼び出すことで、パン、ズーム、スクロールに対応した図を簡単に作成できます。

In [28]:
alt.Chart(cars).mark_point().encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin'
).interactive()

alt.Chart(...)

`tooltip`という視覚変数を使用することでマウスカーソルがあたった時の情報を指定することができます。

In [29]:
alt.Chart(cars).mark_point().encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin',
    tooltip=['Name', 'Origin']
).interactive()

alt.Chart(...)

> Toolchipに馬力が表示されるように指定してみましょう。

In [30]:
# your code goes here

より高度な機能として、リンクされた図やクロスフィルタリングなどの複雑なインタラクションが用意されています。

少し複雑ですが、以下の例では、上段のヒストグラムは年間の自動車台数を示しており、年数を選択することで、馬力と走行距離の関係を示す下段の散布図の点の不透明度が変わり、各年の傾向を全体の中に位置づけて確認することができます。

In [31]:
# create an interval selection over an x-axis encoding
brush = alt.selection_interval(encodings=['x'])

# determine opacity based on brush
opacity = alt.condition(brush, alt.value(0.9), alt.value(0.1))

# an overview histogram of cars per year
# add the interval brush to select cars over time
overview = alt.Chart(cars).mark_bar().encode(
    alt.X('Year:O', timeUnit='year', # extract year unit, treat as ordinal
      axis=alt.Axis(title=None, labelAngle=0) # no title, no label angle
    ),
    alt.Y('count()', title=None), # counts, no axis title
    opacity=opacity
).add_selection(
    brush      # add interval brush selection to the chart
).properties(
    width=400, # set the chart width to 400 pixels
    height=50  # set the chart height to 50 pixels
).interactive()

# a detail scatterplot of horsepower vs. mileage
# modulate point opacity based on the brush selection
detail = alt.Chart(cars).mark_point().encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    # set opacity based on brush selection
    opacity=opacity
).properties(width=400).interactive() # set chart width to match the first chart

# vertically concatenate (vconcat) charts using the '&' operator
overview & detail

alt.VConcatChart(...)

---

これでAltair入門は完了です。

次の演習で、Altairを使ったエンコーディングをより詳しく見ていきましょう。